In [ ]:
# 매일 아침 8시 (UTC+9) 반복 실행
## 데이터베이스에 있던 이전 주식이 반복되지 않아야 함

import pandas as pd
import yfinance as yf
from supabase import create_client, Client
from datetime import datetime, timedelta
import os

# ⓪ API 키 설정 (환경 변수에서 키를 가져오는 것이 안전합니다)
# 로컬 테스트 시: supabase_url = "YOUR_URL"
# 실제 자동화 시: os.environ.get("SUPABASE_URL")
# supabase_url = os.environ.get("SUPABASE_URL")
# supabase_key = os.environ.get("SUPABASE_KEY")

supabase: Client = create_client(supabase_url, supabase_key)

# 한국 주식 목록
Name_KR=['Samsung', 'SK Hynix', 'Samsung Biologics', 'LG Energy Solution', 'Hyundai', 'KB Financial Group', 'Celltrion', 'Doosan Enerbility', 'Kia', 'Hanwha Aerospace', 'Naver', 'HD Hyundai Heavy Industries', 'Shinhan Financial Group', 'Hyundai Mobis', 'Kakao', 'HMM', 'Hana Financial Group', 'POSCO', 'Korea Electric Power', 'Samsung Life Insurance', 'Hanwha Ocean (DSME)', 'HD Korea Shipbuilding & Offshore Engineering', 'SK Square', 'Meritz Financial Group', 'Alteogen', 'Hyundai Rotem Company', 'LG Chem', 'SK Innovation', 'Woori Financial Group', 'Krafton', 'HD Hyundai Electric', 'Industrial Bank of Korea (IBK)', 'Korea Zinc', 'Samsung SDI', 'KakaoBank', 'KT Corporation', 'KT&G (Korea Tobacco)', 'Samsung Heavy Industries', 'LG Electronics', 'LG Corp', 'SK Telecom', 'HYBE', 'SK Group', 'LIG Nex1', 'POSCO Chemical', 'Kakao Pay', 'Samsung Electro-Mechanics', 'Hyundai Glovis', 'Ecopro BM', 'Hanwha Systems', 'Samyang Foods', 'HANMI Semiconductor', 'Doosan', 'Korean Air Lines', 'Posco International', 'HD Hyundai Marine Solution', 'Korea Aerospace Industries', 'Amorepacific', 'Yuhan', 'S-OIL', 'Coway', 'Ecopro', 'Hanwha Solutions', 'Doosan Bobcat', 'Hankook Tire', 'Netmarble', 'LG Household & Health Care', 'LG Display', 'LS Corp.', 'ORION', 'Korea Gas', 'CJ Group', 'NCsoft', 'Kangwon Land', 'LEENO Industrial', 'F&F Co', 'SM Entertainment', 'Kumho Petrochemical', 'Hanmi Pharmaceutical', 'LOTTE Chemical', 'SHIFT UP', 'Pearl Abyss', 'JYP Entertainment', 'KCC Corp', 'Nongshim', 'Fila', 'BGF Retail', 'Pan Ocean', 'LOTTE Corporation', 'Asiana Airlines', 'Hansol Chemical', 'Shinsegae', 'YG Entertainment', 'Doosan Fuel Cell', 'Kakao Games', 'HiteJinro', 'Kumho Tire', 'Ottogi', 'SD BioSensor', 'DoubleUGames']
Ticker_KR=['005930.KS', '000660.KS', '207940.KS', '373220.KS', 'HYMTF', 'KB', '068270.KS', '034020.KS', '000270.KS', '012450.KS', '035420.KS', '329180.KS', 'SHG', '012330.KS', '035720.KS', '011200.KS', '086790.KS', 'PKX', 'KEP', '032830.KS', '042660.KS', '009540.KS', '402340.KS', '138040.KS', '196170.KQ', '064350.KS', '051910.KS', '096775.KS', 'WF', '259960.KS', '267260.KS', '024110.KS', '010130.KS', '006405.KS', '323410.KS', 'KT', '033780.KS', '010140.KS', '066570.KS', '003550.KS', 'SKM', '352820.KS', '034730.KS', '079550.KS', '003670.KS', '377300.KS', '009155.KS', '086280.KS', '247540.KQ', '272210.KS', '003230.KS', '042700.KS', '000150.KS', '003495.KS', '047050.KS', '443060.KS', '047810.KS', '090430.KS', '000100.KS', '010950.KS', '021240.KS', '086520.KQ', '009830.KS', '241560.KS', '161390.KS', '251270.KS', '051905.KS', 'LPL', '006260.KS', '271560.KS', '036460.KS', '001040.KS', '036570.KS', '035250.KS', '058470.KQ', '383220.KS', '041510.KQ', '011780.KS', '008930.KS', '011170.KS', '462870.KS', '263750.KQ', '035900.KQ', '002380.KS', '004370.KS', '081660.KS', '282330.KS', '028670.KS', '004990.KS', '020560.KS', '014680.KS', '004170.KS', '122870.KQ', '336260.KS', '293490.KQ', '000080.KS', '073240.KS', '007310.KS', '137310.KS', '192080.KS']
# 미국 주식 목록
Name_US=['NVIDIA', 'Microsoft', 'Apple', 'Amazon', 'Alphabet (Google)', 'Meta Platforms (Facebook)', 'Broadcom', 'Berkshire Hathaway', 'Tesla', 'JPMorgan Chase', 'Walmart', 'Visa', 'Eli Lilly', 'Oracle', 'Netflix', 'Mastercard', 'Exxon Mobil', 'Costco', 'Procter & Gamble', 'Johnson & Johnson', 'Home Depot', 'Bank of America', 'AbbVie', 'Palantir', 'Coca-Cola', 'UnitedHealth', 'Philip Morris International', 'Cisco', 'T-Mobile US', 'Wells Fargo', 'IBM', 'General Electric', 'Salesforce', 'Chevron', 'Abbott Laboratories', 'Morgan Stanley', 'American Express', 'AMD', 'Walt Disney', 'Goldman Sachs', 'Intuit', 'ServiceNow', 'McDonald', 'AT&T', 'Merck', 'Texas Instruments', 'Uber', 'Intuitive Surgical', 'RTX', 'Blackstone Group', 'Caterpillar', 'Booking Holdings (Booking.com)', 'Pepsico', 'Verizon', 'QUALCOMM', 'BlackRock', 'Charles Schwab', 'Citigroup', 'Boeing', 'S&P Global', 'Thermo Fisher Scientific', 'Adobe', 'Amgen', 'Honeywell', 'Boston Scientific', 'Progressive', 'Applied Materials', 'Nextera Energy', 'Stryker Corporation', 'Danaher', 'Pfizer', 'Capital One', 'Union Pacific Corporation', 'GE Vernova', 'Deere & Company (John Deere)', 'TJX Companies', 'Gilead Sciences', 'Micron Technology', 'Palo Alto Networks', 'Comcast', 'Arista Networks', 'KKR & Co.', 'CrowdStrike', "Lowe's Companies", 'Lam Research', 'Automatic Data Processing', 'KLA', 'Analog Devices', 'Amphenol', 'ConocoPhillips', 'Vertex Pharmaceuticals', 'AppLovin', 'Strategy \n (MicroStrategy)', 'Nike', 'Lockheed Martin', 'Starbucks', 'Marsh & McLennan Companies', 'Intercontinental Exchange', 'American Tower', 'DoorDash']
Ticker_US=['NVDA', 'MSFT', 'AAPL', 'AMZN', 'GOOG', 'META', 'AVGO', 'BRK-B', 'TSLA', 'JPM', 'WMT', 'V', 'LLY', 'ORCL', 'NFLX', 'MA', 'XOM', 'COST', 'PG', 'JNJ', 'HD', 'BAC', 'ABBV', 'PLTR', 'KO', 'UNH', 'PM', 'CSCO', 'TMUS', 'WFC', 'IBM', 'GE', 'CRM', 'CVX', 'ABT', 'MS', 'AXP', 'AMD', 'DIS', 'GS', 'INTU', 'NOW', 'MCD', 'T', 'MRK', 'TXN', 'UBER', 'ISRG', 'RTX', 'BX', 'CAT', 'BKNG', 'PEP', 'VZ', 'QCOM', 'BLK', 'SCHW', 'C', 'BA', 'SPGI', 'TMO', 'ADBE', 'AMGN', 'HON', 'BSX', 'PGR', 'AMAT', 'NEE', 'SYK', 'DHR', 'PFE', 'COF', 'UNP', 'GEV', 'DE', 'TJX', 'GILD', 'MU', 'PANW', 'CMCSA', 'ANET', 'KKR', 'CRWD', 'LOW', 'LRCX', 'ADP', 'KLAC', 'ADI', 'APH', 'COP', 'VRTX', 'APP', 'MSTR', 'NKE', 'LMT', 'SBUX', 'MMC', 'ICE', 'AMT', 'DASH']


# 날짜 계산 (어제 날짜)
today = datetime.now()
yesterday = today - timedelta(days=1)
# yfinance는 end 날짜를 포함하지 않으므로, end를 today로 설정
start_date = yesterday.strftime('%Y-%m-%d')
end_date = today.strftime('%Y-%m-%d')


# API 호출
## 1. 한국 주식
korea_history = []
for i in range(len(Name_KR)):
    name = Name_KR[i]
    Ticker = Ticker_KR[i]
    ticker = yf.Ticker(Ticker)
    

    hist = ticker.history(start=start_date, end=end_date)
    
    if not hist.empty:
        hist['name'] = name
        hist['Ticker'] = Ticker
        korea_history.append(hist)

if korea_history:
    #결과 데이터프레임 생성
    korea_df = pd.DataFrame()
    korea_df = pd.concat(korea_history)

    #전처리
    korea_df_formatted = korea_df.reset_index()
    # yfinance에서 Datetime -> Date로 컬럼명이 바뀔 수 있으므로 확인
    if 'Datetime' in korea_df_formatted.columns:
        date_column_name = 'Datetime'
    else:
        date_column_name = 'Date'

    korea_df_formatted = korea_df_formatted[[date_column_name, 'Ticker', 'name', 'Close', 'Volume']]
    korea_df_formatted = korea_df_formatted.rename(columns={
        date_column_name: 'time',
        'Ticker': 'ticker',
        'name': 'company_name',
        'Close': 'close_price',
        'Volume': 'volume'
    })

    #업로드용 데이터 변환
    data_to_upload_kr = korea_df_formatted.to_dict('records')
    #시간 형식 변경
    for record in data_to_upload_kr:
        record['time'] = record['time'].isoformat()

## 2. 미국 주식
us_history = []
for i in range(len(Name_US)):
    name = Name_US[i]
    Ticker = Ticker_US[i]
    ticker = yf.Ticker(Ticker)
    
    # 'period' 대신 'start'와 'end'를 사용하고, 하루치 데이터이므로 'interval="1d"' 사용
    hist = ticker.history(start=start_date, end=end_date)
    
    if not hist.empty:
        hist['name'] = name
        hist['Ticker'] = Ticker
        us_history.append(hist)
    # 데이터 삽입 (upsert 사용)
    # 기본 키(time, ticker)가 겹치면 데이터를 덮어쓰고, 없으면 새로 추가합니다.
    try:
        data, count = supabase.table('korean_stocks').upsert(data_to_upload_kr).execute()
        print(f"✅ 성공적으로 업로드/업데이트되었습니다. 처리된 행 개수: {len(data[1])}")
    except Exception as e:
        print(f"🚨 업로드 중 오류가 발생했습니다: {e}")

if us_history:
    #결과 데이터프레임 생성
    us_df = pd.DataFrame()
    us_df = pd.concat(us_history)

    #전처리
    us_df_formatted = us_df.reset_index()
    # yfinance에서 Datetime -> Date로 컬럼명이 바뀔 수 있으므로 확인
    if 'Datetime' in us_df_formatted.columns:
        date_column_name = 'Datetime'
    else:
        date_column_name = 'Date'

    us_df_formatted = us_df_formatted[[date_column_name, 'Ticker', 'name', 'Close', 'Volume']]
    us_df_formatted = us_df_formatted.rename(columns={
        date_column_name: 'time',
        'Ticker': 'ticker',
        'name': 'company_name',
        'Close': 'close_price',
        'Volume': 'volume'
    })

    #업로드용 데이터 변환
    data_to_upload_us = us_df_formatted.to_dict('records')
    #시간 형식 변경
    for record in data_to_upload_us:
        record['time'] = record['time'].isoformat()

    # 데이터 삽입 (upsert 사용)
    # 기본 키(time, ticker)가 겹치면 데이터를 덮어쓰고, 없으면 새로 추가합니다.
    try:
        data, count = supabase.table('us_stocks').upsert(data_to_upload_us).execute()
        print(f"✅ 성공적으로 업로드/업데이트되었습니다. 처리된 행 개수: {len(data[1])}")
    except Exception as e:
        print(f"🚨 업로드 중 오류가 발생했습니다: {e}")
else:
    print("ℹ️ 어제는 주식 시장이 열리지 않았거나, 가져올 데이터가 없습니다.")